In [1]:
# All import statements
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno
from bs4 import BeautifulSoup
from datetime import datetime
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import pickle
from nltk.stem import PorterStemmer 
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
from tqdm import tqdm
from prettytable import PrettyTable
tqdm.pandas()

C:\ProgramData\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
df = pd.read_csv('train.csv')
synonmys = pd.read_csv('synonyms.csv',names=['act','exp'])
replace_dict = pickle.load(open('replace.pkl','rb'))

In [109]:
x = PrettyTable()
x.field_names=['actual','replacement']
for key,value in replace_dict.items():
    x.add_row([key,value])
print(x)

+------------------------------+--------------------------+
|            actual            |       replacement        |
+------------------------------+--------------------------+
|         nutri system         |       nutrisystem        |
|         soda stream          |        sodastream        |
|        playstation's         |            ps            |
|         playstations         |            ps            |
|         playstation          |            ps            |
|            (ps 2)            |           ps2            |
|            (ps 3)            |           ps3            |
|            (ps 4)            |           ps4            |
|             ps 2             |           ps2            |
|             ps 3             |           ps3            |
|             ps 4             |           ps4            |
|         coffeemaker          |       coffee maker       |
|            k-cups            |          k cup           |
|            k-cup             |        

In [4]:
synonmys.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   act     51 non-null     object
 1   exp     51 non-null     object
dtypes: object(2)
memory usage: 944.0+ bytes


In [5]:
df.head()

,id,query,product_title,product_description,median_relevance,relevance_variance
0,1,bridal shower decorations,Accent Pillow with Heart Design - Red/Black,Red satin accent pillow embroidered with a hea...,1,0.000
1,2,led christmas lights,Set of 10 Battery Operated Multi LED Train Chr...,Set of 10 Battery Operated Train Christmas Lig...,4,0.000
2,4,projector,ViewSonic Pro8200 DLP Multimedia Projector,NaN,4,0.471
3,5,wine rack,Concept Housewares WR-44526 Solid-Wood Ceiling...,"Like a silent and sturdy tree, the Southern En...",4,0.000
4,7,light bulb,Wintergreen Lighting Christmas LED Light Bulb ...,"WTGR1011\nFeatures\nNickel base, 60,000 averag...",2,0.471


In [6]:
df.product_description.fillna('',inplace=True)

In [7]:
df.head(2)

,id,query,product_title,product_description,median_relevance,relevance_variance
0,1,bridal shower decorations,Accent Pillow with Heart Design - Red/Black,Red satin accent pillow embroidered with a hea...,1,0.0
1,2,led christmas lights,Set of 10 Battery Operated Multi LED Train Chr...,Set of 10 Battery Operated Train Christmas Lig...,4,0.0


In [8]:
synonmys.head(2)

,act,exp
0,girls,girl
1,girl's,girl


In [9]:
a = synonmys['act'].astype(str)

In [10]:
b = synonmys['exp'].astype(str)

In [11]:
changeto = []
for i in b:
    changeto.append(i)

In [12]:
actual = []
for i in a:
    actual.append(i)

## Remove HTML tags from the product description column

In [13]:
def cleanHTML(phrase):
    soup = BeautifulSoup(phrase,'lxml')
    text = soup.get_text()
    return text

In [14]:
def word_replacement(text):
    
    for act,exp in zip(actual,changeto):
        text = re.sub(act,exp,text)
    for key,value in replace_dict.items():
        text = re.sub(str(key),str(value),text)
        
    return text
    

In [15]:
def decontraction(phrase):
        # Decontraction
    
    # specific
    phrase = phrase.lower()
    
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"\’t|\'t", " not", phrase)
    phrase = re.sub(r"\’re|'re", " are", phrase)
    phrase = re.sub(r"’s|\'s", " is", phrase)
    phrase = re.sub(r"’d|\'d", " would", phrase)
    phrase = re.sub(r"’ll|\'ll", " will", phrase)
    phrase = re.sub(r"’ve|\'ve", " have", phrase)
    phrase = re.sub(r"’m|\'m", " am", phrase)
    
    return phrase

In [16]:
stop_words = stopwords.words('english')
stop_words.remove('not')
stop_words.remove('no')

In [17]:
token_pattern = r"\b\w\w+\b"
p = PorterStemmer()
def stemming(text):
    token = re.compile(token_pattern, flags = re.UNICODE)
    tokens = [x.lower() for x in token.findall(text)]
    out = []
    for token in tokens:
        out.append(p.stem(token))
    
    out = ' '.join([word for word in out])
    return out

In [136]:
stemming('batteries')

'batteri'

In [18]:
# https://stackoverflow.com/a/47091490/4084039

def text_preprocess(phrase,remove_html = True, decontract = True,stem=False):
    
    #Removing URL from sentences
    phrase = re.sub(r'http/S+','',phrase)
#     print('After removing URL',phrase)
    
    if remove_html:
        phrase = cleanHTML(phrase)
#         print('Cleaned HTML',phrase)
    
    # The below step will take more time
    phrase = word_replacement(phrase)
#     print('word replaced',phrase)

    if stem:
        phrase = stemming(phrase)
#         print('stemming',phrase)
    if decontract:
        phrase = decontraction(phrase)
#         print('decontraction',phrase)
    
    #Remove everything except alphabets, numbers,spaces
    phrase = re.sub(r'[^0-9a-zA-Z ]+',' ',phrase)
#     print('remove alpha num',phrase)
    
#     # Removing words which has numbers in them    
#     phrase = re.sub(r'[a-zA-z]+\d\S*', " ", phrase)
#     print('remove words with numbers',phrase)
    
    # Removing all new line characters and double or more spaces together
    phrase = re.sub(r"\n", " ", phrase)    
#     print('remove new line',phrase)
    phrase = re.sub(' +', " ", phrase)
#     print('remove extra space',phrase)
    
    # Removing stop words and words which have length>1
    words = phrase.split()
    output = []
    for word in words:
        if word not in stop_words and len(word)>1:
            output.append(word)
#     print('Output array',output)
    output = ' '.join(word for word in output)
    
    return output

In [98]:
start = datetime.now()
df['cleaned_query1'] = df['query'].str.lower()
df['cleaned_query1'] = df.apply(lambda x : text_preprocess(x['cleaned_query1'],stem=True),axis = 1)
end = datetime.now()
print('Time take to run this cell is',end-start)

Time take to run this cell is 0:00:04.521481


In [20]:
# Replacing space with underscore to make it a single word.
start = datetime.now()
df['cleaned_query'] = df.apply(lambda x : x['query'].replace(' ','_'),axis = 1)
end = datetime.now()
print('Time take to run this cell is',end-start)

Time take to run this cell is 0:00:00.092535


In [21]:
start = datetime.now()
df['cleaned_product_title'] = df.apply(lambda x : text_preprocess(x['product_title']),axis = 1)
end = datetime.now()
print('Time take to run this cell is',end-start)

Time take to run this cell is 0:00:04.478391


In [22]:
start = datetime.now()
df['cleaned_product_description'] = df.apply(lambda x : text_preprocess(x['product_description']),axis = 1)
end = datetime.now()
print('Time take to run this cell is',end-start)

C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:424: MarkupResemblesLocatorWarning: "http://i104.photobucket.com/albums/m175/champions_on_display/wincraft2013/januaryb/65497012.jpg" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning
C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:424: MarkupResemblesLocatorWarning: "http://i104.photobucket.com/albums/m175/champions_on_display/wincraft2013/januaryb/65516012.jpg" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning


Time take to run this cell is 0:00:05.491938


C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:424: MarkupResemblesLocatorWarning: "http://i104.photobucket.com/albums/m175/champions_on_display/wincraft2013/januaryb/6552101" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning


In [23]:
## With stemming for 

In [24]:
start = datetime.now()
df['ste_cleaned_product_title'] = df.apply(lambda x : text_preprocess(x['product_title'],stem=True),axis = 1)
end = datetime.now()
print('Time take to run this cell is',end-start)

Time take to run this cell is 0:00:05.851493


In [25]:
start = datetime.now()
df['ste_cleaned_product_description'] = df.apply(lambda x : text_preprocess(x['product_description'],stem=True),axis = 1)
end = datetime.now()
print('Time take to run this cell is',end-start)

Time take to run this cell is 0:00:17.097202


In [26]:
df['product_description_len'] = df.ste_cleaned_product_title.str.len()

In [27]:
df['product_title_len'] = df.ste_cleaned_product_description.str.len()

In [28]:
df.head()

,id,query,product_title,product_description,median_relevance,relevance_variance,cleaned_query1,cleaned_query,cleaned_product_title,cleaned_product_description,ste_cleaned_product_title,ste_cleaned_product_description,product_description_len,product_title_len
0,1,bridal shower decorations,Accent Pillow with Heart Design - Red/Black,Red satin accent pillow embroidered with a hea...,1,0.000,bridal shower decor,bridal_shower_decorations,accent pillow heart design red black,red satin accent pillow embroidered heart blac...,accent pillow heart design red black,red satin accent pillow embroid heart black th...,36,50
1,2,led christmas lights,Set of 10 Battery Operated Multi LED Train Chr...,Set of 10 Battery Operated Train Christmas Lig...,4,0.000,led christma light,led_christmas_lights,set 10 battery operated multi led train christ...,set 10 battery operated train christmas lights...,set 10 batteri oper multi led train christma l...,set 10 batteri oper train christma light item ...,61,643
2,4,projector,ViewSonic Pro8200 DLP Multimedia Projector,,4,0.471,projector,projector,viewsonic pro8200 dlp multimedia projector,,viewson pro8200 dlp multimedia projector,,40,0
3,5,wine rack,Concept Housewares WR-44526 Solid-Wood Ceiling...,"Like a silent and sturdy tree, the Southern En...",4,0.000,wine rack,wine_rack,concept housewares wr 44526 solid wood ceiling...,like silent sturdy tree southern enterprises b...,concept housewar wr 44526 solid wood ceil wall...,like silent sturdi tree southern enterpris bir...,82,454
4,7,light bulb,Wintergreen Lighting Christmas LED Light Bulb ...,"WTGR1011\nFeatures\nNickel base, 60,000 averag...",2,0.471,light bulb,light_bulb,wintergreen lighting christmas led light bulb ...,wtgr1011 features nickel base 60 000 average h...,wintergreen light christma led light bulb pack 25,wtgr1011 featur nickel base 60 000 averag hour...,49,262


In [29]:
start = datetime.now()
df['cleaned_query1'] = df['query'].str.lower()
df['cleaned_query1'] = df.apply(lambda x : text_preprocess(x['cleaned_query1'],stem=True),axis = 1)
end = datetime.now()
print('Time take to run this cell is',end-start)

Time take to run this cell is 0:00:04.126861


In [30]:
df['query_product_title_words_overlap_nopre'] = np.zeros(df.shape[0])
for index, query_text in enumerate(df['query']):
    count = 0
    query_text = query_text.lower().split()
    product_title = df.product_title[index].lower() 
    for word in query_text:
        if word not in product_title:
            df.loc[index,'query_product_title_words_overlap_nopre'] = 0
            break
    else:
        df.loc[index,'query_product_title_words_overlap_nopre'] = 1
    

In [31]:
df['query_product_title_words_overlap'] = np.zeros(df.shape[0])
for index, query_text in enumerate(df['cleaned_query1']):
    count = 0
    query_text = query_text.lower().split()
    product_title = df.ste_cleaned_product_title[index].lower() 
    for word in query_text:
        if word not in product_title:
            df.loc[index,'query_product_title_words_overlap'] = 0
            break
    else:
        df.loc[index,'query_product_title_words_overlap'] = 1
    

In [32]:
with_pre = df.groupby(['query_product_title_words_overlap','median_relevance'])['query_product_title_words_overlap'].describe()





In [33]:
without_pre = df.groupby(['query_product_title_words_overlap_nopre','median_relevance'])['query_product_title_words_overlap_nopre'].describe()





In [34]:
without_pre

count  mean  std  \
query_product_title_words_overlap_nopre median_relevance                      
0.0                                     1                  706.0   0.0  0.0   
                                        2                 1036.0   0.0  0.0   
                                        3                  953.0   0.0  0.0   
                                        4                 2034.0   0.0  0.0   
1.0                                     1                   68.0   1.0  0.0   
                                        2                  440.0   1.0  0.0   
                                        3                  784.0   1.0  0.0   
                                        4                 4137.0   1.0  0.0   

                                                          min  25%  50%  75%  \
query_product_title_words_overlap_nopre median_relevance                       
0.0                                     1                 0.0  0.0  0.0  0.0   
                                        2                 0.0  0.0  0.0  0.0   
                                        3                 0.0  0.0  0.0  0.0   
                                        4                 0.0  0.0  0.0  0.0   
1.0                                     1                 1.0  1.0  1.0  1.0   
                                        2                 1.0  1.0  1.0  1.0   
                                        3                 1.0  1.0  1.0  1.0   
                                        4                 1.0  1.0  1.0  1.0   

                                                          max  
query_product_title_words_overlap_nopre median_relevance       
0.0                                     1                 0.0  
                                        2                 0.0  
                                        3                 0.0  
                                        4                 0.0  
1.0                                     1                 1.0  
                                        2                 1.0  
                                        3                 1.0  
                                        4                 1.0

In [35]:
rows = df.shape[0]
x = PrettyTable()
print('Table showing the percentage of points which have 100% match between query and title')
x.field_names = ['Median relevancy',"with preprocessing(%)", "without preprocessing(%)"]
for score in [1,2,3,4]:
    x.add_row([score,round(with_pre['count'][1][score]/rows,2)*100,round(without_pre['count'][1][score]/rows,2)*100])

print(x)
    

Table showing the percentage of points which have 100% match between query and title
+------------------+-----------------------+--------------------------+
| Median relevancy | with preprocessing(%) | without preprocessing(%) |
+------------------+-----------------------+--------------------------+
|        1         |          1.0          |           1.0            |
|        2         |          5.0          |           4.0            |
|        3         |          9.0          |           8.0            |
|        4         |          49.0         |           41.0           |
+------------------+-----------------------+--------------------------+


In [101]:
df = df.drop('query_product_title_words_overlap_nopre',axis = 1)

In [102]:
df.head(2)

,id,query,product_title,product_description,median_relevance,relevance_variance,cleaned_query1,cleaned_query,cleaned_product_title,cleaned_product_description,ste_cleaned_product_title,ste_cleaned_product_description,product_description_len,product_title_len,query_product_title_words_overlap
0,1,bridal shower decorations,Accent Pillow with Heart Design - Red/Black,Red satin accent pillow embroidered with a hea...,1,0.0,bridal shower decor,bridal_shower_decorations,accent pillow heart design red black,red satin accent pillow embroidered heart blac...,accent pillow heart design red black,red satin accent pillow embroid heart black th...,36,50,0.0
1,2,led christmas lights,Set of 10 Battery Operated Multi LED Train Chr...,Set of 10 Battery Operated Train Christmas Lig...,4,0.0,led christma light,led_christmas_lights,set 10 battery operated multi led train christ...,set 10 battery operated train christmas lights...,set 10 batteri oper multi led train christma l...,set 10 batteri oper train christma light item ...,61,643,1.0


In [106]:
df.shape

(10158, 15)

In [105]:
pickle.dump(df,open('data/main_df.pkl','wb'))

## Train - CV - test split

In [36]:
df.shape

(10158, 16)

In [37]:
df_train,df_test = train_test_split( df, test_size=0.15, random_state=42)
df_train,df_cv = train_test_split( df_train, test_size=0.15, random_state=42)

In [38]:
df_train.shape

(7338, 16)

In [39]:
df_cv.shape, df_test.shape

((1296, 16), (1524, 16))

In [40]:
pickle.dump((df_train, df_cv, df_test),open('data/dataframes.pkl','wb'))

In [41]:
df_train, df_cv, df_test = pickle.load(open('data/dataframes.pkl','rb'))

## Vectorizing Query data

In [42]:
df_train['cleaned_query'].value_counts()

wireless_mouse         81
memory_foam_pillow     78
rachel_ray_cookware    61
bike_lock              58
16_gb_memory_card      52
                       ..
8_ounce_mason_jars      9
fifa_15                 8
donut_shoppe_k_cups     7
polo_bear_sweater       6
dollhouse_bathtub       6
Name: cleaned_query, Length: 261, dtype: int64

In [43]:
vectorizer = CountVectorizer()
vectorizer.fit(df_train['cleaned_query'])
query_train = vectorizer.fit_transform(df_train['cleaned_query'])
query_cv = vectorizer.transform(df_cv['cleaned_query'])
query_test = vectorizer.transform(df_test['cleaned_query'])

CountVectorizer()

In [44]:
query_train = vectorizer.fit_transform(df_train['cleaned_query'])
query_cv = vectorizer.transform(df_cv['cleaned_query'])
query_test = vectorizer.transform(df_test['cleaned_query'])


In [45]:
query_train.shape,query_cv.shape

((7338, 261), (1296, 261))

In [46]:
pickle.dump((query_train, query_cv, query_test),open('data/query_ohe.pkl','wb'))

In [47]:
# query_train, query_cv, query_test = pickle.load(open('data/query_ohe.pkl','rb'))

### TODO -- <font color ='red'> Try Response coding of categorical features

## Vectorizing the product title with TFIDF vectorizer


In [120]:
tfidf_product_title_vectorizer = TfidfVectorizer(min_df=3,max_df=300)
product_title_tfidf_train = tfidf_product_title_vectorizer.fit_transform(df_train['ste_cleaned_product_title'])
product_title_tfidf_cv = tfidf_product_title_vectorizer.transform(df_cv['ste_cleaned_product_title'])
product_title_tfidf_test = tfidf_product_title_vectorizer.transform(df_test['ste_cleaned_product_title'])

In [121]:
product_title_tfidf_train.shape,product_title_tfidf_test.shape

((7338, 2903), (1524, 2903))

In [50]:
pickle.dump((product_title_tfidf_train, product_title_tfidf_cv, product_title_tfidf_test),open('data/product_title_tfidf.pkl','wb'))

In [51]:
# product_title_tfidf_train, product_title_tfidf_cv, product_title_tfidf_test = pickle.load(open('data/product_title_tfidf.pkl','rb'))

In [52]:
pickle.dump((tfidf_product_title_vectorizer),open('data/product_title_vectorizer.pkl','wb'))

In [53]:
# tfidf_product_title_vectorizer = pickle.load(open('data/product_title_vectorizer.pkl','rb'))

## Vectorizing the product title with TFIDF vectorizer with uni, bi and tri grams


In [128]:
vectorizer = TfidfVectorizer(min_df=3,ngram_range=(1,3),max_df=500,max_features=500)
product_title_tfidf_ngram_train = vectorizer.fit_transform(df_train['ste_cleaned_product_title'])
product_title_tfidf_ngram_cv = vectorizer.transform(df_cv['ste_cleaned_product_title'])
product_title_tfidf_ngram_test = vectorizer.transform(df_test['ste_cleaned_product_title'])

In [129]:
product_title_tfidf_ngram_train.shape,product_title_tfidf_ngram_test.shape

((7338, 500), (1524, 500))

In [130]:
pickle.dump((product_title_tfidf_ngram_train, product_title_tfidf_ngram_cv, product_title_tfidf_ngram_test),open('data/product_title_tfidf_ngram.pkl','wb'))

In [57]:
# product_title_tfidf_ngram_train, product_title_tfidf_ngram_cv, product_title_tfidf_ngram_test = pickle.load(open('data/product_title_tfidf_ngram.pkl','rb'))

### Vectorizing the product title with W2V vectorizer

In [58]:
def load_glove_model(path):
    #Loading the glove vectors
    with open(path, 'rb') as f:
        model = pickle.load(f)
        glove_words = set(model.keys())
    return model,glove_words

In [59]:
def get_W2V(column):
    model,glove_words = load_glove_model('glove_vectors')
    output = []
    check = 0
    for sentence in column:
        # Intializing the vector with all zeroes
        mean_vector = np.zeros(300)
        count = 0
        for word in sentence.split():
            # For each word check the presence in glove words
            if word in glove_words:
                # Get the vector corresponding to that word
                vector = model.get(word)
                # Add all the vectors
                mean_vector+=vector
                count+=1
        # Finding the mean
        if(count!=0):
            mean_vector = mean_vector/count
        else:
            check+=1

        output.append(mean_vector)
    print('All zero vectors are:',check)
    output = np.array(output)
    return output


In [60]:
product_title_w2v_train = get_W2V(df_train.cleaned_product_title)
product_title_w2v_cv    = get_W2V(df_cv.cleaned_product_title)
product_title_w2v_test  = get_W2V(df_test.cleaned_product_title)

All zero vectors are: 4
All zero vectors are: 0
All zero vectors are: 1


In [61]:
pickle.dump((product_title_w2v_train, product_title_w2v_cv, product_title_w2v_test),open('data/product_title_w2v.pkl','wb'))

In [62]:
# product_title_w2v_train, product_title_w2v_cv, product_title_w2v_test= pickle.load(open('data/product_title_w2v.pkl','rb'))

In [63]:
product_title_w2v_train.shape,product_title_w2v_test.shape

((7338, 300), (1524, 300))

### Vectorizing product title with TFIDF- W2V vectorizer

In [64]:
tfidf_word_index_mappingdict = dict([(word,index) for index,word in enumerate(tfidf_product_title_vectorizer.get_feature_names())])

In [65]:
def get_w2v_tfidf(column,vectorizer,tfidf_matrix):
    
    model,glove_words = load_glove_model('glove_vectors')
    
    tfidf_words = vectorizer.get_feature_names()
    output = []
    check = 0
    for index,sentence in enumerate(column):

        mean_vector = np.zeros(300)
        count = 0

        for word in sentence.split():
            
            if word in glove_words and word in tfidf_words:
            
                word_index =  tfidf_word_index_mappingdict[word]
                tfidf_value = tfidf_matrix[index].toarray()[:,word_index]
                if(tfidf_value>0):            
                    vector = model.get(word)*tfidf_value
                    mean_vector+=vector
                    count+=1
        
        if(count>0):                                    
            mean_vector = mean_vector/count        
        else:
            check+=1
        output.append(mean_vector)
    print('All zero vectors:',check)
    output = np.array(output)
    return output
                
            
    

In [66]:
product_title_tfidf_w2v_train = get_w2v_tfidf(df_train.cleaned_product_title,
                                              tfidf_product_title_vectorizer,
                                              product_title_tfidf_train)

All zero vectors: 102


In [67]:
product_title_tfidf_w2v_cv = get_w2v_tfidf(df_cv.cleaned_product_title,
                                              tfidf_product_title_vectorizer,
                                              product_title_tfidf_cv)

All zero vectors: 18


In [68]:
product_title_tfidf_w2v_test = get_w2v_tfidf(df_test.cleaned_product_title,
                                              tfidf_product_title_vectorizer,
                                              product_title_tfidf_test)

All zero vectors: 25


In [69]:
pickle.dump((product_title_tfidf_w2v_train, product_title_tfidf_w2v_cv, product_title_tfidf_w2v_test),open('data/product_title_tfidf_w2v.pkl','wb'))

In [70]:
# product_title_tfidf_w2v_train, product_title_tfidf_w2v_cv, product_title_tfidf_w2v_test= pickle.load(open('data/product_title_tfidf_w2v.pkl','rb'))

## Vectorizing the product description with TFIDF vectorizer.

In [71]:
tfidf_product_description_vectorizer = TfidfVectorizer(min_df=3)
product_description_tfidf_train = tfidf_product_description_vectorizer.fit_transform(df_train['ste_cleaned_product_description'])
product_description_tfidf_cv    = tfidf_product_description_vectorizer.transform(df_cv['ste_cleaned_product_description'])
product_description_tfidf_test  = tfidf_product_description_vectorizer.transform(df_test['ste_cleaned_product_description'])

In [72]:
product_description_tfidf_train.shape

(7338, 6666)

In [73]:
pickle.dump((product_description_tfidf_train, product_description_tfidf_cv, product_description_tfidf_test),open('data/product_description_tfidf.pkl','wb'))

In [74]:
# product_description_tfidf_train, product_description_tfidf_cv, product_description_tfidf_test= pickle.load(open('data/product_description_tfidf.pkl','rb'))

In [75]:
pickle.dump((tfidf_product_description_vectorizer),open('data/tfidf_product_description_vectorizer.pkl','wb'))

In [76]:
# tfidf_product_description_vectorizer = pickle.load(open('data/tfidf_product_description_vectorizer.pkl','rb'))

## Vectorizing the product description with TFIDF vectorizer and data with uni,bi and tri grams.

In [131]:
tfidf_ngram_product_description_vectorizer = TfidfVectorizer(min_df=3,ngram_range=(1,2),max_features=500)
product_description_tfidf_ngram_train = tfidf_ngram_product_description_vectorizer.fit_transform(df_train['ste_cleaned_product_description']+df_train['ste_cleaned_product_title'])
product_description_tfidf_ngram_cv    = tfidf_ngram_product_description_vectorizer.transform(df_cv['ste_cleaned_product_description']+df_cv['ste_cleaned_product_title'])
product_description_tfidf_ngram_test  = tfidf_ngram_product_description_vectorizer.transform(df_test['ste_cleaned_product_description']+df_test['ste_cleaned_product_title'])

In [132]:
product_description_tfidf_ngram_train.shape

(7338, 500)

In [133]:
pickle.dump((product_description_tfidf_ngram_train, product_description_tfidf_ngram_cv, product_description_tfidf_ngram_test),open('data/product_description_tfidf_ngram.pkl','wb'))

In [80]:
# product_description_tfidf_ngram_train, product_description_tfidf_ngram_cv, product_description_tfidf_ngram_test= pickle.load(open('data/product_description_tfidf_ngram.pkl','rb'))

In [81]:
pickle.dump((tfidf_ngram_product_description_vectorizer),open('data/tfidf_ngram_product_description_vectorizer.pkl','wb'))

In [82]:
# tfidf_product_description_vectorizer = pickle.load(open('data/tfidf_product_description_vectorizer.pkl','rb'))

### Vectorizing the product description with W2V vectorizer

In [83]:
product_description_w2v_train = get_W2V(df_train.cleaned_product_description)
product_description_w2v_cv    = get_W2V(df_cv.cleaned_product_description)
product_description_w2v_test  = get_W2V(df_test.cleaned_product_description)

All zero vectors are: 1785
All zero vectors are: 307
All zero vectors are: 376


In [84]:
product_description_w2v_train.shape, product_description_w2v_test.shape

((7338, 300), (1524, 300))

In [85]:
pickle.dump((product_description_w2v_train, product_description_w2v_cv, product_description_w2v_test),open('data/product_description_w2v.pkl','wb'))

In [86]:
# product_description_w2v_train, product_description_w2v_cv, product_description_w2v_test= pickle.load(open('data/product_description_w2v.pkl','rb'))

### Vectorizing the product description with TFIDF - W2V 

In [87]:
tfidf_word_index_mappingdict = dict([(word,index) for index,word in enumerate(tfidf_product_description_vectorizer.get_feature_names())])

In [88]:
product_description_tfidf_w2v_train = get_w2v_tfidf(df_train.cleaned_product_description,
                                              tfidf_product_description_vectorizer,
                                              product_description_tfidf_train)

All zero vectors: 1787


In [89]:
product_description_tfidf_w2v_cv = get_w2v_tfidf(df_cv.cleaned_product_description,
                                              tfidf_product_description_vectorizer,
                                              product_description_tfidf_cv)

All zero vectors: 308


In [90]:
product_description_tfidf_w2v_test = get_w2v_tfidf(df_test.cleaned_product_description,
                                              tfidf_product_description_vectorizer,
                                              product_description_tfidf_test)

All zero vectors: 376


In [91]:
pickle.dump((product_description_tfidf_w2v_train, product_description_tfidf_w2v_cv, product_description_tfidf_w2v_test),open('data/product_description_tfidf_w2v.pkl','wb'))

In [92]:
# product_description_tfidf_w2v_cv, product_description_tfidf_w2v_cv, product_description_tfidf_w2v_test= pickle.load(open('data/product_description_tfidf_w2v.pkl','rb'))

In [93]:
df.head(2)

,id,query,product_title,product_description,median_relevance,relevance_variance,cleaned_query1,cleaned_query,cleaned_product_title,cleaned_product_description,ste_cleaned_product_title,ste_cleaned_product_description,product_description_len,product_title_len,query_product_title_words_overlap_nopre,query_product_title_words_overlap
0,1,bridal shower decorations,Accent Pillow with Heart Design - Red/Black,Red satin accent pillow embroidered with a hea...,1,0.0,bridal shower decor,bridal_shower_decorations,accent pillow heart design red black,red satin accent pillow embroidered heart blac...,accent pillow heart design red black,red satin accent pillow embroid heart black th...,36,50,0.0,0.0
1,2,led christmas lights,Set of 10 Battery Operated Multi LED Train Chr...,Set of 10 Battery Operated Train Christmas Lig...,4,0.0,led christma light,led_christmas_lights,set 10 battery operated multi led train christ...,set 10 battery operated train christmas lights...,set 10 batteri oper multi led train christma l...,set 10 batteri oper train christma light item ...,61,643,1.0,1.0
